# Solving the diffusion approximation to the dimensionless birth-immigration-death process

<!-- <center><font size="+4">Solving the diffusion approximation to the birth-immigration-death process</font></center> -->

## Forward Laplace transform

In [1]:
diffusioneqn (rho, s, M) := 
    diff (rho, tau) -
    (1/(2*s)) * diff (x * rho, x, 2) +
    diff (x * rho, x, 1) +
    M * diff (rho, x);

                                                    1
(%o1) diffusioneqn(rho, s, M) := diff(rho, tau) - (---) diff(x rho, x, 2)
                                                   2 s
                                           + diff(x rho, x, 1) + M diff(rho, x)

In [2]:
cgfeqn (gam, s, M) := 
    diff (gam, tau) +
    ((1/(2*s)) * theta^2 - theta) * diff (gam, theta) -
    M * theta;

(%o2) cgfeqn(gam, s, M) := diff(gam, tau)
                             1        2
                        + ((---) theta  - theta) diff(gam, theta) + (- M) theta
                            2 s

In [3]:
factor(diffusioneqn (rho0(x,tau), s, M));

             2
            d                            d
(%o3) - (x (--- (rho0(x, tau))) - 2 s x (-- (rho0(x, tau)))
              2                          dx
            dx
          d                       d
 - 2 M s (-- (rho0(x, tau))) + 2 (-- (rho0(x, tau)))
          dx                      dx
         d
 - 2 s (---- (rho0(x, tau))) - 2 s rho0(x, tau))/(2 s)
        dtau

## Characteristic equation

Compute the integral for the implicit solution.

In [4]:
implicitint : logcontract(integrate(1/(thp^2/(2*s) -  thp), thp));

                                    thp - 2 s
(%o4)                           log(---------)
                                       thp

In [5]:
factor(diff(implicitint, thp));

                                      2 s
(%o5)                           ---------------
                                thp (thp - 2 s)

In [6]:
logcontract(subst(theta, thp, implicitint) -
    subst(theta0, thp, implicitint));

                               (theta - 2 s) theta0
(%o6)                      log(--------------------)
                               theta (theta0 - 2 s)

In [7]:
:lisp (msetq $implicitsol '((MTIMES SIMP) ((MPLUS SIMP) ((MTIMES SIMP) -2 $S ) $THETA) $THETA0 ((MEXPT SIMP) ((MPLUS SIMP) ((MTIMES SIMP) -2 $S) $THETA0) -1) ((MEXPT SIMP) $THETA -1)))

((MTIMES SIMP) ((MPLUS SIMP) ((MTIMES SIMP) -2 $S) $THETA) $THETA0
 ((MEXPT SIMP) ((MPLUS SIMP) ((MTIMES SIMP) -2 $S) $THETA0) -1)
 ((MEXPT SIMP) $THETA -1))

In [8]:
logcontract(subst(theta, thp, implicitint) -
    subst(theta0, thp, implicitint) -
    log(implicitsol));

(%o8)                                  0

Solve the implicit equation.

In [9]:
thetasol : solve ([implicitsol = %e^(tau)], [theta])[1];
theta0sol : solve ([implicitsol = %e^(tau)], [theta0])[1];

                                       2 s theta0
(%o9)              theta = - ------------------------------
                                tau                     tau
                             (%e    - 1) theta0 - 2 s %e

                                          tau
                                    2 s %e    theta
(%o10)                 theta0 = -----------------------
                                   tau
                                (%e    - 1) theta + 2 s

Check the solution.

In [10]:
subst(0, tau,  thetasol);

(%o11)                          theta = theta0

In [11]:
chareqn(theta) := diff(theta, tau) - (1/(2*s)) * theta^2 + theta;

                                                  1        2
(%o12)     chareqn(theta) := diff(theta, tau) - (---) theta  + theta
                                                 2 s

In [12]:
factor(chareqn(part(thetasol, 2)));

(%o13)                                 0

## Homogeneous solution

Re-expressing $u$ in terms of $\tau$ and including the initial condition $P(x,0) = \delta(x-x_0) \Rightarrow \Gamma_{\mathrm{hom}}(\theta, 0)=x_0 \, \theta$, where $\Rightarrow$ indicates taking the logarithm of the Laplace transform, yields

In [13]:
gamhom : x0 * part(theta0sol, 2);

                                    tau
                              2 s %e    theta x0
(%o14)                      -----------------------
                               tau
                            (%e    - 1) theta + 2 s

Noting $M=0$ in the homogeneous case

In [14]:
factor(cgfeqn (gamhom, s, 0));

(%o15)                                 0

## Inhomogeneous solutions

Find the characteristic which passes through a given point (theta_f, tau_f).

In [15]:
th0sol : solve([thetaf = 
        subst(tauf, tau, part(thetasol, 2))], [theta0])[1];

                                         tauf
                                   2 s %e     thetaf
(%o16)                theta0 = -------------------------
                                  tauf
                               (%e     - 1) thetaf + 2 s

In [16]:
thfsol : factor(psubst(th0sol, thetasol));

                                        tauf
                                  2 s %e     thetaf
(%o17)         theta = ----------------------------------------
                         tauf            tau                tau
                       %e     thetaf - %e    thetaf + 2 s %e

In [17]:
subst(tauf, tau, thfsol);

(%o18)                          theta = thetaf

Integrate along that characteristic.

In [18]:
ginhint : integrate(part(thfsol, 2), tau);

                                - tauf
             tauf         tau %e
(%o19) 2 s %e     thetaf (------------
                             thetaf
                              - tauf       tau                    tauf
                            %e       log(%e    (thetaf - 2 s) - %e     thetaf)
                          - --------------------------------------------------)
                                                  thetaf

In [19]:
ginhint : integrate((thetaf*s*%e^(uf*s))/
    (s*%e^(s*u)-D*thetaf*%e^(s*u)+D*thetaf*%e^(uf*s)), u);

                            - s uf
                  s uf  u %e
(%o20) s thetaf %e     (----------
                         D thetaf
                            - s uf                      s u              s uf
                          %e       log((D thetaf - s) %e    - D thetaf %e    )
                        - ----------------------------------------------------)
                                               D s thetaf

Check the solution.

In [20]:
factor(cgfeqn (psubst([tauf = tau, thetaf = theta], gaminh), s, M));

(%o21)                             - M theta

In [21]:
gaminh : M *logcontract(ev(factor(subst(tauf, tau, ginhint) - 
            subst(0, tau, ginhint)), logexpand=super));

(%o22)                                 0

## Inverse Laplace transform

### Homogeneous solution

We proceed by constructing a change of variable to place the inverse Laplace transform integral into a form enabling the identification of a Bessel function solution. Combining the form of the Laplace transform with the form of `gamhom` we have

In [22]:
schemintgr : x * theta  - a * theta / (b * theta + 1);

                                         a theta
(%o23)                       theta x - -----------
                                       b theta + 1

The following change of variable $\theta \rightarrow \theta'$ will render `shemintgr` a function of $\theta' + \frac{1}{\theta'}$ as expected for a Bessel function

In [23]:
expand(subst(c * thp - 1/b, theta, schemintgr));

                                    x      a       a
(%o24)                    c thp x - - + -------- - -
                                    b    2         b
                                        b  c thp

Since we are looking for a form that includes a single factor of $\theta' + \frac{1}{\theta'}$ we look for $c x = \frac{a}{b^2 c}$

In [24]:
expand(subst((a^(1/2) * x^(-1/2) * thp - 1)/b, theta, 
            schemintgr));

                  x    sqrt(a) thp sqrt(x)   sqrt(a) sqrt(x)   a
(%o25)         (- -) + ------------------- + --------------- - -
                  b             b                 b thp        b

To provide $\theta$ in terms of $\theta'$ we substitute the values of $a$ and $b$ given by the homogeneous solution `gamhom`. First we note that if we define $a = x_0  e^{\tau}$ and $b = \frac{1}{2s} (e^{s \tau} -1)$

In [25]:
aquant : x0 * %e^(tau);
bquant : (%e^(tau) - 1) / (2 * s);

                                     tau
(%o26)                             %e    x0

                                     tau
                                   %e    - 1
(%o27)                             ---------
                                      2 s

In [26]:
homthetasub: factor(psubst([a = aquant, b = bquant],
(a^(1/2) * x^(-1/2) * thp - 1)/b));

                            tau/2
                     2 s (%e      thp sqrt(x0) - sqrt(x))
(%o28)               ------------------------------------
                                tau
                             (%e    - 1) sqrt(x)

In [27]:
thetarecipcoeff : factor(psubst([a = aquant, b = bquant],
(a^(1/2) * x^(-1/2) /b)));

                                   tau/2
                             2 s %e      sqrt(x0)
(%o29)                       --------------------
                                tau
                             (%e    - 1) sqrt(x)

In [28]:
thetaexpconst : factor(psubst([a = aquant, b = bquant],
-a/b - x/b));

                                      tau
                               2 s (%e    x0 + x)
(%o30)                       - ------------------
                                     tau
                                   %e    - 1

### Inhomogeneous solution

In [29]:
invinhvarchange : theta = 
    thetap/x - 2 * s / (%e^(tau) - 1);

                                  thetap      2 s
(%o31)                    theta = ------ - ---------
                                    x        tau
                                           %e    - 1

In [30]:
factor(psubst(invinhvarchange, 
  (%e^(tau) - 1) * theta / (2 * s) + 1));

                                 tau
                              (%e    - 1) thetap
(%o32)                        ------------------
                                    2 s x

In [31]:
inhdifsol : x^(2*M*s - 1) * 
    %e^(-2*s*x/(%e^(tau) - 1)) /
    ((%e^(tau) - 1)^(2*M*s));

                                            2 s x
                                        - ---------
                                            tau
                            2 M s - 1     %e    - 1
                           x          %e
(%o33)                     ------------------------
                                  tau     2 M s
                               (%e    - 1)

In [32]:
factor(diffusioneqn (inhdifsol, s, M));

(%o34)                                 0